In [1]:
!pip install datamol
!pip install rdkit-pypi
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.7/381.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 20.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Project/ALVS")

Mounted at /content/drive


In [3]:
import pandas as pd
from rdkit import Chem
import glob
import datamol as dm
from sklearn.model_selection import train_test_split

In [4]:
def load_score(mol_files):
    name_scores = []
    for mol_file in mol_files:
        mols = Chem.SDMolSupplier(mol_file, sanitize=False)
        for m in mols:
            score = m.GetProp("minimizedAffinity")
            name = m.GetProp("_Name")
            try:
                smi = Chem.MolToSmiles(m)
            except:
                print(m)
            smi = smi.replace("/", "").replace("@", "").replace("\\", "")
            name_scores.append([name, smi, float(score)])
    return name_scores

In [5]:
def ensemble_merge(df):
    name_miniscore = []
    for name, dfg in df.groupby("name"):
        mini_score = dfg["score"].min()
        smis = dfg["smiles"].unique()
        # if len(smis) != 1:
        #     print(name)
        #     raise RuntimeError("smiles is not unique!")
        smi = smis[0]
        name_miniscore.append([name, smi, mini_score])
    df_merge = pd.DataFrame(name_miniscore)
    df_merge.columns = ["name", "smiles", "score"]
    return df_merge

In [6]:
def fix_smiles(smi):
    mol = dm.to_mol(smi, ordered=True, sanitize=False)
    mol = dm.fix_mol(mol)
    mol = dm.sanitize_mol(mol, sanifix=True, charge_neutral=False)
    if not mol:
        return "error"
    mol = dm.standardize_mol(mol, disconnect_metals=False, normalize=True, reionize=False, uncharge=False)
    smi = dm.standardize_smiles(dm.to_smiles(mol))
    return smi

In [7]:
mol_files = glob.glob("data/dpp4/*.sdf")

In [8]:
name_scores = load_score(mol_files)

[19:42:11] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****



[19:42:21] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****



[19:42:44] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****



[19:43:15] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****



[19:43:24] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****



[19:43:44] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****



In [9]:
df = pd.DataFrame(name_scores)
df.columns = ["name", "smiles", "score"]

In [10]:
df_merge = ensemble_merge(df)

In [11]:
len(df_merge)

41443

In [12]:
df_merge.head()

,name,smiles,score
0,CHEMBL1082462,[H]N([H])(CC(=O)N1C[CH](F)C[CH]1C#N)C(C)(C)CC(...,-7.204528
1,CHEMBL1082653,[H]N([H])(CC(=O)N1CCC[CH]1C#N)[CH]1C[CH]2C[CH]...,-6.820651
2,CHEMBL1082707,[H]N([H])(CC(=O)N1CCC[CH]1C#N)[CH]1C[CH]2CN(C(...,-7.730747
3,CHEMBL1082708,[H]N([H])(CC(=O)N1CCC[CH]1C#N)[CH]1C[CH]2CN(C(...,-7.715799
4,CHEMBL1082716,[H]N([H])(CC(=O)N1CCC[CH]1C#N)[CH]1C[CH]2CN(C(...,-7.631647


In [13]:
df_merge["smiles"] = df_merge["smiles"].map(lambda x: fix_smiles(x))

Streaming output truncated to the last 5000 lines.
[19:48:16] Initializing Normalizer
[19:48:16] Running Normalizer
[19:48:16] WARNING: not removing hydrogen atom without neighbors
[19:48:16] Initializing Normalizer
[19:48:16] Running Normalizer
[19:48:16] Initializing MetalDisconnector
[19:48:16] Running MetalDisconnector
[19:48:16] Initializing Normalizer
[19:48:16] Running Normalizer
[19:48:16] Explicit valence for atom # 24 N, 4, is greater than permitted
[19:48:16] DEBUG: CC1=CC(C2=NNC=C2C[NH]2CCCC(C3=CC=NN3)C2)=C(C)C=C1 failed due to Explicit valence for atom # 24 N, 4, is greater than permitted

[19:48:16] Explicit valence for atom # 11 N, 4, is greater than permitted
[19:48:16] WARNING: not removing hydrogen atom without neighbors
[19:48:16] Explicit valence for atom # 24 N, 4, is greater than permitted
[19:48:16] Explicit valence for atom # 24 N, 4, is greater than permitted
[19:48:16] INFO: [ValenceValidation] Explicit valence for atom # 24 N, 4, is greater than permitted
[19

In [14]:
df_merge.head()

,name,smiles,score
0,CHEMBL1082462,CC(C)(CC(=O)N1CCCC1)NCC(=O)N1CC(F)CC1C#N,-7.204528
1,CHEMBL1082653,COC1CC2CC(NCC(=O)N3CCCC3C#N)CC2C1,-6.820651
2,CHEMBL1082707,N#CC1CCCN1C(=O)CNC1CC2CN(C(=O)N3CCCC3)CC2C1,-7.730747
3,CHEMBL1082708,N#CC1CCCN1C(=O)CNC1CC2CN(C(=O)N3CCCCC3)CC2C1,-7.715799
4,CHEMBL1082716,N#CC1CCCN1C(=O)CNC1CC2CN(C(=O)N3CCOCC3)CC2C1,-7.631647


In [15]:
df_train, df_test = train_test_split(df_merge, test_size=0.4)

In [16]:
df_train.to_csv("preprocess_data/dpp4/dpp4_train.csv", index=None, sep="\t")
df_test.to_csv("preprocess_data/dpp4/dpp4_test.csv", index=None, sep="\t")

In [17]:
df_train.head()

,name,smiles,score
20531,ZINC39698571,N#CC1=C(N)OC2c3ccccc3NC(=O)C2C1c1ccsc1,-8.466708
6450,ZINC12024393,COC(CNC(=O)c1ccc(N2C(=O)C(C)CS2(=O)=O)cc1Cl)OC,-7.515980
15104,ZINC35773834,Cc1c(N)cccc1S(=O)(=O)N1CCOCC1C,-6.889817
15636,ZINC36298376,CNCc1ccccc1NS(=O)(=O)c1ccc(F)cc1Cl,-7.383318
15587,ZINC36271999,CCN1CCN(Cc2c(-c3ccc4c(c3)CCCC4)nc3ccc(N)cn23)CC1,-9.963980


In [18]:
len(df_train)

24865

In [19]:
len(df_test)

16578

In [20]:
df_merge.to_csv("preprocess_data/dpp4/dpp4_processed.csv", index=None, sep="\t")